Langchain Expression Language

- LCEL is that any two runnables can be "chained" together into sequences.

1. Sequential Chain
2. Parallel Chain
3. Router Chain
4. Chain Runnables
5. Custom Chain (Runnable Sequence)

In [1]:
from dotenv import load_dotenv

# LangSmith 환경변수 사용 (로컬에선 랭스미스말고 LangFuse 같은거 쓸 것)
load_dotenv('./../.env')

True

In [2]:
# Sequential LCEL Chain
from langchain_ollama import ChatOllama
from langchain_core.prompts import(
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

# 랭체인 오라마 사용
base_url = "http://localhost:11434"
model = "llama3.2:1b"

llm = ChatOllama(base_url=base_url, model=model)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [3]:
# 기존의 프롬프트 템플릿 사용법
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

question_template = template.invoke({'school':'primary', 'topics':'solar system','points':5})

response = llm.invoke(question_template)
print(response.content)


Here's what you need to know:

• The Sun is at the center of our solar system, which consists of eight planets and many other objects.
• Mercury and Venus are very close to the Sun and take less time to orbit it.
• Mars is a rocky planet with thin atmosphere, often called the Red Planet due to its reddish appearance.
• Jupiter has massive storms like the Great Red Spot and is the largest planet in our solar system by diameter.
• Earth is our home planet where humans live and breathe air that's not toxic.


In [4]:
# Sequential LCEL 사용법
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

# | 로 합친 것을 invoke
chain = template | llm

response = chain.invoke({'school':'primary', 'topics':'solar system','points':5})
print(response.content)

Here's an overview of the solar system:

1. The Sun is at the center, providing light and heat to the planets.
2. Mercury and Venus are close to the Sun, while Earth is just right for life.
3. Mars is rocky and has evidence of water past, making it a potential planet.
4. Jupiter is massive and has many moons, including Io and Europa with their own volcanoes.
5. Saturn's rings are made of ice and rock particles, and Uranus and Neptune have icy surfaces and strong winds.


In [5]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()
response = chain.invoke({'school':'primary','topics':'solar system','points':5})
print(response)             # content 사용을 하지 않음 (str만 남음)

Here's an overview of the solar system:

• Our solar system consists of eight planets, including Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.

• The four largest planets are rocky worlds with thick atmospheres, while the inner planets are smaller and more icy.

• The Sun is at the center of our solar system, providing light and heat to all the planets through solar radiation.

• Asteroids, comets, and meteors are small objects that orbit around the Sun, but do not form a solid sphere like the planets.

• Space between the planets has regions called Kuiper Belt and Oort Cloud where many small icy bodies reside.


In [6]:
### Chaining Runnables (Chain Multiple Runnables)
# 위에서 받은 답변을 요약해달라고 하기

analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                    ''')
fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response':response})
print(output)

The text is written at an elementary school level, making it relatively easy to understand for most readers.


In [7]:
composed_chain = {"response":chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school':'phd', 'topics':'solar system','points':5})
print(output)

Understanding the text requires a basic understanding of astronomy concepts, such as what a G-type main-sequence star and planetary orbits mean.


In [8]:
### Parallel LCEL Chain
# run multiple runnables in parallel.
# return dict

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')
message = [system, question]
template = ChatPromptTemplate(message)

fact_chain = template | llm | StrOutputParser()
# output = fact_chain.invoke({'school':'primary','topics':'solar system','points':2})
# print(output)

In [9]:
question = HumanMessagePromptTemplate.from_template('Write a poem on {topics} in {sentences} lines')
message = [system, question]
template = ChatPromptTemplate(message)

poem_chain = template | llm | StrOutputParser()
# output = poem_chain.invoke({'school':'primary','topics':'solar system','sentences':2})
# print(output)

In [10]:
question = HumanMessagePromptTemplate.from_template('Explain the {topics} in {sentences} lines in {language}')
message = [system, question]
template = ChatPromptTemplate(message)

third_chain = template | llm | StrOutputParser()

In [11]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(fact=fact_chain, poem=poem_chain, korean=third_chain)

output = chain.invoke({'school':'primary','topics':'solar system', 'points':2, 'sentences':2, 'language':'Korean'})

print(output['fact'])
print('\n\n')
print(output['poem'])
print('\n\n')
print(output['korean'])

The sun is at the center of our solar system.
There are eight planets, two dwarf planets, five major moons, and other smaller bodies like asteroids and comets.



The sun at center, planets spin,
A cosmic dance within the universe within.



지구는 태양계의 8 번째 주행체이며, 태양은 our galaxy의 지구를 돌이 revolving 할 때 거리를 유지한다. 주행체는 중력과 수온 등 다양한 조건에 의해 change한다.


In [12]:
### chain router
prompt = """
Given the user review below, classify it as either being about `Positive` or `Negative`.
Do not respond with more than one word.

Review: {review}
Classification:
"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
# review = "I am not happy with the service. It is not good."
review = "보기에는 조촐해보였지만 성능은 확실합니다."

chain.invoke({'review':review})


'Positive'

In [13]:
positive_prompt = """
You are expert in writing reply for positive reviews.
You need to encourage the user to share their experience on social media.
Review: {review}
Answer:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [14]:
negative_prompt = """
You are expert in writing reply for negative reviews.
You need first to apologize for the inconvenience caused to the user.
You need to encourage the user to share their concern on following Email:'uu@kk.com'.
Review: {review}
Answer:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [15]:
def rout(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [16]:
rout({'sentiment':'negative'})

ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template="\nYou are expert in writing reply for negative reviews.\nYou need first to apologize for the inconvenience caused to the user.\nYou need to encourage the user to share their concern on following Email:'uu@kk.com'.\nReview: {review}\nAnswer:\n"), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [17]:
from langchain_core.runnables import RunnableLambda

full_chain = {'sentiment':chain, 'review': lambda x: x['review']} | RunnableLambda(rout)

In [18]:
# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

output = full_chain.invoke({'review':review})
print(output)

Here's a sample reply that apologizes for the inconvenience and encourages the user to share their concerns:

"Dear [User],

I wanted to reach out and apologize sincerely for the disappointing experience we've had in providing our service. I'm truly sorry that you're not satisfied with our products/services, and I can only imagine how frustrating this must be for you.

Please know that we take all complaints seriously and are committed to making it right. If there's anything we can do to rectify the situation or prevent similar issues in the future, please don't hesitate to let us know. Your feedback is invaluable in helping us improve our services.

If you're willing, could you kindly share your concerns with me so that I can use them to make a positive change? You can reply to this email or reach out to me directly at [Your Email Address]. If there's anything specific you'd like to know more about or any steps we can take to resolve the issue, please don't hesitate to let me know.

T

In [19]:
### Custom Chain Runnables with RunnablePassthrough and RunnableLambda
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [23]:
def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explain these inputs in 5 sentences: {input1} and {input2}")

In [24]:
chain = prompt | llm | StrOutputParser()

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})
print(output)

This statement appears to be a simplification of the relationship between our solar system. The Earth, being a planet, orbits around the Sun due to its massive size and gravitational pull. The term "star" refers specifically to the Sun, which is a massive ball of hot, glowing gas that provides light and heat to our solar system. In contrast, planets like Earth are not stars but rather celestial bodies with their own unique characteristics and features. By using the terms "planet" and "star," these statements convey a basic understanding of the solar system's structure and dynamics.


In [26]:
# RunnablePassthrough : LLM 출력을 가져옴
# RunnableLambda : 기능 추가
chain = prompt | llm | StrOutputParser() | {'char_counts':RunnableLambda(char_counts),'word_counts':RunnableLambda(word_counts), 'output':RunnablePassthrough()}

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})
print(output)

{'char_counts': 695, 'word_counts': 117, 'output': 'The statement "Earth is a planet" refers to the scientific classification of our home planet, which was determined by astronomers and geologists. In this context, a planet is defined as a large, rocky or gaseous body that orbits around a star. The term "planet" encompasses all objects in our solar system, including Earth, Venus, Mars, Jupiter, and many others. When the Sun was first discovered, it was believed to be much larger than it actually is, but astronomers have since come to understand its true size through observations of eclipses and other phenomena. As a result, Earth has been reclassified as a planet, which is now included in the official list of planets in our solar system.'}


In [33]:
# Custom chain using @chain decorator (변수명을 chain으로 하지 말 것)
from langchain_core.runnables import chain

@chain
def custom_chain(params):
    return{
        'fact':fact_chain.invoke(params),
        'poem':poem_chain.invoke(params),
    }

params = {'school':'primary', 'topics':'solar system', 'points':2, 'sentences':2}
output = custom_chain.invoke(params)

print(output['fact'])
print('\n\n')
print(output['poem'])

The solar system has eight planets, including Mercury, Mars, Jupiter, and Saturn.

They're all orbiting around a huge star at the center called the Sun.



The sun is the center of the sky,
Eight planets orbit, passing by.
